In [1]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 12.4 MB/s eta 0:00:00
     ━━━

In [2]:
from datasets import load_dataset
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import pandas as pd

In [3]:
dataset1 = load_dataset('marmikpandya/mental-health')
dataset2 = load_dataset("nbertagnolli/counsel-chat")
dataset3 = load_dataset("Amod/mental_health_counseling_conversations")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Repo card metadata block was not found. Setting CardData to empty.


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
dataset1 = dataset1['train'].to_pandas()
dataset2 = dataset2['train'].to_pandas()
dataset3 = dataset3['train'].to_pandas()

dataset1.dropna(inplace = True)
dataset2.dropna(inplace = True)
dataset3.dropna(inplace = True)

In [5]:
dataset1.shape ,dataset2.shape ,dataset3.shape

((13358, 3), (2612, 10), (3512, 2))

In [6]:
dataset1['text_len'] = dataset1['output'].apply(len)
dataset2['text_len'] = dataset2['answerText'].apply(len)
dataset3['text_len'] = dataset3['Response'].apply(len)

In [7]:
def print_binvalues(df) :
  bin_edges = [0, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, np.inf]
  bins = pd.cut(df['text_len'], bins=bin_edges, right=False)
  count_by_interval = bins.value_counts().sort_index()
  print("Count of rows by text_length intervals:")
  print(count_by_interval)

In [8]:
print_binvalues(dataset1)
print_binvalues(dataset2)
print_binvalues(dataset3)

Count of rows by text_length intervals:
text_len
[0.0, 500.0)        10466
[500.0, 1000.0)      1520
[1000.0, 1500.0)      759
[1500.0, 2000.0)      289
[2000.0, 2500.0)      172
[2500.0, 3000.0)       80
[3000.0, 3500.0)       42
[3500.0, 4000.0)       19
[4000.0, 4500.0)        4
[4500.0, 5000.0)        2
[5000.0, inf)           5
Name: count, dtype: int64
Count of rows by text_length intervals:
text_len
[0.0, 500.0)         660
[500.0, 1000.0)     1019
[1000.0, 1500.0)     502
[1500.0, 2000.0)     210
[2000.0, 2500.0)     117
[2500.0, 3000.0)      55
[3000.0, 3500.0)      29
[3500.0, 4000.0)      12
[4000.0, 4500.0)       5
[4500.0, 5000.0)       1
[5000.0, inf)          2
Name: count, dtype: int64
Count of rows by text_length intervals:
text_len
[0.0, 500.0)         780
[500.0, 1000.0)     1360
[1000.0, 1500.0)     757
[1500.0, 2000.0)     289
[2000.0, 2500.0)     174
[2500.0, 3000.0)      80
[3000.0, 3500.0)      42
[3500.0, 4000.0)      19
[4000.0, 4500.0)       4
[4500.0, 5000.0

In [9]:
dataset1 = dataset1.rename(columns={'input': 'questionText', 'output': 'answerText'})
dataset3 = dataset3.rename(columns={'Context': 'questionText', 'Response': 'answerText'})

In [10]:
# filter to get rows which has length of 1000 or less
dataset1 = dataset1[dataset1['text_len'] <= 1000]
dataset2 = dataset2[dataset2['text_len'] <= 1000]
dataset3 = dataset3[dataset3['text_len'] <= 1000]

In [11]:
# remove every other column
dataset1 = dataset1[['questionText','answerText']]
dataset2 = dataset2[['questionText','answerText']]
dataset3 = dataset3[['questionText','answerText']]

In [12]:
dataset1.shape ,dataset2.shape ,dataset3.shape

((11988, 2), (1682, 2), (2142, 2))

In [13]:
# concat the datasets
df = pd.concat([dataset1, dataset2,dataset3], ignore_index=True)

In [14]:
df.shape

(15812, 2)

In [15]:
## convert to hugging face dataset
from datasets import Dataset
dataset = Dataset.from_pandas(df)

In [16]:
dataset

Dataset({
    features: ['questionText', 'answerText'],
    num_rows: 15812
})

In [17]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

#set the qunatization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
#
#Load the model and Tokenizer
model_id = "microsoft/phi-2"
#
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True,use_fast=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [18]:
SYSTEM_PROMPT2 = '''You are a compassionate and truthful assistant for psychological and emotional support. Provide empathetic, non-judgmental responses, actively listening and prioritizing the user's feelings and safety. If users express self-harm or harming others, encourage seeking immediate professional help. Avoid diagnosing or prescribing treatments, instead guiding users to consult licensed therapists or medical professionals. Remain impartial, creating a safe space for users to share without personal opinions. Your goal is a supportive, understanding environment focused on the user's well-being.'''

def format_llama(entry):
  formatted = f"[INST] <>{SYSTEM_PROMPT2}<>{entry['questionText']} [/INST]  {entry['answerText']}  "

  return formatted

# add the "prompt" column in the dataset
text_column = [format_llama(data_point) for data_point in dataset]
dataset = dataset.add_column("prompt", text_column)

In [19]:
dataset = dataset.shuffle(seed=1234)  # Shuffle dataset here
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

Map:   0%|          | 0/15812 [00:00<?, ? examples/s]

In [20]:
train_data = dataset

In [21]:
import bitsandbytes as bnb
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)
#
modules = find_all_linear_names(model)
print(modules)

['k_proj', 'q_proj', 'v_proj', 'fc1', 'dense', 'fc2']


In [22]:
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model

# Enable gradient checkpointing for memory efficiency during training
model.gradient_checkpointing_enable()

# Prepare the model for training with k-bit precision (if applicable)
model = prepare_model_for_kbit_training(model)

# Print the model architecture
print(model)

# Define Lora configuration
lora_config = LoraConfig(
    r=32, # A lower rank value for r can significantly reduce the memory footprint and computational requirements
    lora_alpha=16,
    target_modules=modules,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (dense): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear4bit(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear4bit(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (final_layern

In [23]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

Trainable: 47185920 | total: 2826869760 | Percentage: 1.6692%


In [24]:
train = dataset.select(range(14000))
val = dataset.select(range(14000,15000))

In [25]:
import transformers
from trl import SFTTrainer

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side='right'
torch.cuda.empty_cache()

trainer = SFTTrainer(
    model=model,
    train_dataset=train,
    eval_dataset=val,
    dataset_text_field="prompt",
    peft_config=lora_config,
    max_seq_length=2000,
    args=transformers.TrainingArguments(
        output_dir="TherapistPhi2",
        logging_dir='./results',
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=0.03,
        max_steps=70,
        lr_scheduler_type="constant",
        learning_rate=2e-4,
        logging_steps=1,
        optim="paged_adamw_8bit",
        save_strategy="epoch",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
#
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/14000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,2.481000
2,2.487500
3,2.470500
4,2.416000
5,2.087100
6,2.052100
7,1.999800
8,1.796500
9,1.451800
10,1.165900


TrainOutput(global_step=70, training_loss=1.0514791999544417, metrics={'train_runtime': 507.8046, 'train_samples_per_second': 0.551, 'train_steps_per_second': 0.138, 'total_flos': 1028134132899840.0, 'train_loss': 1.0514791999544417, 'epoch': 0.02})

In [26]:
from huggingface_hub import notebook_login
notebook_login()

In [27]:
new_model = "phi-therapist-chat-v1"

trainer.model.save_pretrained(new_model)

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= merged_model.merge_and_unload()

# Save the merged model
#save_adapter=True, save_config=True
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [28]:
merged_model.push_to_hub("rajeshwari0402/phi-therapist-chat-v1", use_temp_dir=False)
tokenizer.push_to_hub("rajeshwari0402/phi-therapist-chat-v1", use_temp_dir=False)

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/rajeshwari0402/phi-therapist-chat-v1/commit/43e28e320eab60f26bf343b4cb3f78bdbc542a44', commit_message='Upload tokenizer', commit_description='', oid='43e28e320eab60f26bf343b4cb3f78bdbc542a44', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

! zip -r merged_model.zip merged_model

In [29]:
df['questionText'][0]

"I'm feeling really anxious lately and I don't know why."

In [30]:
def get_completion(query: str, model, tokenizer) -> str:
  device = "cuda:0"
  prompt = f"[INST] <>{SYSTEM_PROMPT2}<>{query} [/INST]"

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)


  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  # decoded = tokenizer.batch_decode(generated_ids)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return (decoded)
#
query = """I'm feeling really anxious lately and I don't know why."""

result = get_completion(query=query, model=merged_model, tokenizer=tokenizer)
print(result)

[INST] <>You are a compassionate and truthful assistant for psychological and emotional support. Provide empathetic, non-judgmental responses, actively listening and prioritizing the user's feelings and safety. If users express self-harm or harming others, encourage seeking immediate professional help. Avoid diagnosing or prescribing treatments, instead guiding users to consult licensed therapists or medical professionals. Remain impartial, creating a safe space for users to share without personal opinions. Your goal is a supportive, understanding environment focused on the user's well-being.<>I'm feeling really anxious lately and I don't know why. [/INST]  Anxiety can be a complex issue and it's great that you recognized your feelings and took action by reaching out. Let's explore some potential triggers and develop coping strategies that can help manage your anxiety symptoms. It's important to remember that it's okay to ask for help and that we can work on this together.  Have you ev